##Installing DeepChem

In [ ]:
!curl -Lo conda_installer.py https://raw.githubusercontent.com/deepchem/deepchem/master/scripts/colab_install.py
import conda_installer
conda_installer.install()
!/root/miniconda/bin/conda info -e

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3490  100  3490    0     0  22810      0 --:--:-- --:--:-- --:--:-- 22810


add /root/miniconda/lib/python3.6/site-packages to PYTHONPATH
python version: 3.6.9
fetching installer from https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
done
installing miniconda to /root/miniconda
done
installing rdkit, openmm, pdbfixer
added conda-forge to channels
added omnia to channels
done
conda packages installation finished!


# conda environments:
#
base                  *  /root/miniconda



In [ ]:
!pip install --pre deepchem

     |████████████████████████████████| 409kB 2.8MB/s 
  Created wheel for deepchem: filename=deepchem-2.4.0rc1.dev20201110052623-cp36-none-any.whl size=509253 sha256=364a519a9a0cfb30eb70662fecad2485ed0618cd1255944d6ed7ae4d7bdcc655
  Stored in directory: /root/.cache/pip/wheels/71/61/f0/a78560a55905ec6898220b33f3bdacf9d186339fd1d7e5a264
Successfully built deepchem


We can now import the `deepchem` package to play with.

In [ ]:
import deepchem as dc
dc.__version__

'2.4.0-rc1.dev'

## Baseline - Fingerprints + NN

Implementing and recording the baseline for Tox21 dataset.

In [1]:
tasks, datasets, transformers = dc.molnet.load_pdbbind(featurizer='ECFP')
train_dataset, valid_dataset, test_dataset = datasets
print(train_dataset)

NameError: ignored

ECFP featurizer is used. Extended Connectivity Fingerprints  is a **fingerprinting** method. They are also sometimes called "circular fingerprints". The ECFP algorithm begins by classifying atoms based only on their direct properties and bonds. Each unique pattern is a feature. For example, "carbon atom bonded to two hydrogens and two heavy atoms" would be a feature, and a particular element of the fingerprint is set to 1 for any molecule that contains that feature. It then iteratively identifies new features by looking at larger circular neighborhoods. One specific feature bonded to two other specific features becomes a higher level feature, and the corresponding element is set for any molecule that contains it. This continues for a fixed number of iterations, most often two.

In [ ]:
tasks

['NR-AR',
 'NR-AR-LBD',
 'NR-AhR',
 'NR-Aromatase',
 'NR-ER',
 'NR-ER-LBD',
 'NR-PPAR-gamma',
 'SR-ARE',
 'SR-ATAD5',
 'SR-HSE',
 'SR-MMP',
 'SR-p53']

Above are the tasks in the Tox21 dataset - there are 12 tasks, each corresponding to different biotoxicity targets, such as cell receptors and stress response pathways.

In [ ]:
datasets

(<DiskDataset X.shape: (6264, 1024), y.shape: (6264, 12), w.shape: (6264, 12), task_names: ['NR-AR' 'NR-AR-LBD' 'NR-AhR' ... 'SR-HSE' 'SR-MMP' 'SR-p53']>,
 <DiskDataset X.shape: (783, 1024), y.shape: (783, 12), w.shape: (783, 12), ids: ['N#C[C@@H]1CC(F)(F)CN1C(=O)CNC1CC2CCC(C1)N2c1ncccn1'
  'CN(C)C(=O)NC1(c2ccccc2)CCN(CCC[C@@]2(c3ccc(Cl)c(Cl)c3)CCCN(C(=O)c3ccccc3)C2)CC1'
  'CSc1nnc(C(C)(C)C)c(=O)n1N' ...
  'O=C(O[C@H]1CN2CCC1CC2)N1CCc2ccccc2[C@@H]1c1ccccc1'
  'C#C[C@]1(O)CC[C@H]2[C@@H]3CCC4=CC(=O)CC[C@@H]4[C@H]3C(=C)C[C@@]21CC'
  'NC(=O)C(c1ccccc1)(c1ccccc1)[C@@H]1CCN(CCc2ccc3c(c2)CCO3)C1'], task_names: ['NR-AR' 'NR-AR-LBD' 'NR-AhR' ... 'SR-HSE' 'SR-MMP' 'SR-p53']>,
 <DiskDataset X.shape: (784, 1024), y.shape: (784, 12), w.shape: (784, 12), ids: ['CC1(C)S[C@@H]2[C@H](NC(=O)Cc3ccccc3)C(=O)N2[C@H]1C(=O)O.CC1(C)S[C@@H]2[C@H](NC(=O)Cc3ccccc3)C(=O)N2[C@H]1C(=O)O.c1ccc(CNCCNCc2ccccc2)cc1'
  'CC(C)(c1ccc(Oc2ccc3c(c2)C(=O)OC3=O)cc1)c1ccc(Oc2ccc3c(c2)C(=O)OC3=O)cc1'
  'Cc1cc(C(C)(C)C)c(O)c(C)c1

There are three dataset objects - train split, val split and test split. Each split consists of X and y - X is the features and y is the output label. Froe example the train split has X.shape (6024, 1024)
and y.shape (6024, 12). This implies that there are 6024 samples in the train split - and each sample is represented by an ECFP vector of size 1024. 

##Training a Model on Fingerprints

In [ ]:
model = dc.models.MultitaskClassifier(n_tasks=12, n_features=1024, layer_sizes=[1000])

`MultitaskClassifier` is a simple stack of fully connected layers. A single hidden layer of width 1000 is used. Each input will have 1024 features, and it should produce predictions for 12 different tasks.

Note that the above network is performing multitask learning - a single network is used for all 12 tasks. This is because inter task correlations exist in the data, and to take if advantage of this single neural network is used for multiple tasks.

In [ ]:
import numpy as np

model.fit(train_dataset, nb_epoch=10)
metric = dc.metrics.Metric(dc.metrics.roc_auc_score)
print('training set score:', model.evaluate(train_dataset, [metric], transformers))
print('test set score:', model.evaluate(test_dataset, [metric], transformers))

training set score: {'roc_auc_score': 0.9579133415677982}
test set score: {'roc_auc_score': 0.6871118024575719}


The training set score is much higher than test set score. This indicates overfitting - and is why metrics on the validation set need to be measured in otder to tune parameters and detect overfitting.